In [1]:
import os
from bs4 import BeautifulSoup 
from collections import defaultdict
import json

    
base_dir = '/Users/mac/Desktop/dss master thisis/bpmn-for-research-master/Text2Process-master/TestData'

all_nodes = []
bpmn_element = defaultdict(list)
for root, dirs, files in os.walk(base_dir):
    for f in files:
        if f.endswith('.model'):
            xml_file = os.path.join(root, f)
#             print(xml_file)
            
            # Reading the data inside the xml file to a variable under the name  data
            with open(xml_file, 'r') as f:
                data = f.read() 

            # Passing the stored data inside the beautifulsoup parser 
            bs_data = BeautifulSoup(data, 'xml') 
            
            nodes = bs_data.find_all('node')
            for node in nodes:
                properties = node.find('property', {'name':'#type'})
                if properties:
                    all_nodes.append(node)
                    k = properties.get('value').split('.')[-1]
                    v = node.find('property', {'name':'text'}).get('value')
                    bpmn_element[k].append(v)
# bpmn_element

bpmn_element = {k:list(set(bpmn_element[k])) for k in bpmn_element}
for k in bpmn_element:
    if '' in bpmn_element[k]:
        bpmn_element[k].remove('')
bpmn_element2 = bpmn_element.copy()
for k in bpmn_element2:
    if bpmn_element2[k] == []:
        del bpmn_element[k]
        
# bpmn_element

bpmn_elements = {}
bpmn_elements['activities'] = bpmn_element['Task']
bpmn_elements['onjects'] = bpmn_element['DataObject']
bpmn_elements['Es'] = bpmn_element['MessageStartEvent']
bpmn_elements['Ee'] = bpmn_element['EndEvent']
bpmn_elements['Ei'] = bpmn_element['MessageIntermediateEvent']
bpmn_elements['roles'] = bpmn_element['Lane']
bpmn_elements['roles'].extend(bpmn_element['Pool'])

# bpmn_elements

with open('./bpmn_elements.json','w') as f:
    json.dump(bpmn_elements,f)